In [58]:
# Calculate Price of Option with martingale pricing method and Girsanov theorem
import numpy as np
import scipy.stats
import math

p = float(input("Enter current price of underlying asset : "))
r = float(input("Enter interest rate : "))
q = float(input("Enter q : "))
sd = float(input("Enter standard deviation : "))
t = float(input("maturity from now (year) : "))
k1 = float(input("strike price 1 : "))
k2 = float(input("strike price 2 : "))
k3 = float(input("strike price 3 : "))
k4 = float(input("strike price 4 : "))

k_l = [k1,k2,k3,k4]
d1_l = []
d2_l = []
nor_d1_l = []
nor_d2_l = []
 # d1 = (math.log(p/k)+(r-q+((sd**2)/2))*t)/(sd*(t**(1/2)))
 # d2 = (math.log(p/k)+(r-q-((sd**2)/2))*t)/(sd*(t**(1/2)))
 # scipy.stats.norm(0, 1).cdf(d1)

 #計算各執行價的 N(d1())、N(d2())
for i in range(4):
    d1_l.append((math.log(p/k_l[i])+(r-q+((sd**2)/2))*t)/(sd*(t**(1/2))))
    d2_l.append((math.log(p/k_l[i])+(r-q-((sd**2)/2))*t)/(sd*(t**(1/2))))
    nor_d1_l.append(scipy.stats.norm(0, 1).cdf((math.log(p/k_l[i])+(r-q+((sd**2)/2))*t)/(sd*(t**(1/2)))))
    nor_d2_l.append(scipy.stats.norm(0, 1).cdf((math.log(p/k_l[i])+(r-q-((sd**2)/2))*t)/(sd*(t**(1/2)))))
#print("nor_d1_l", nor_d1_l)
#print("nor_d2_l", nor_d2_l)
#計算各段payoff的期望現值
value_1 = p*np.exp(-q*t)*(nor_d1_l[0]-nor_d1_l[1])-k1*np.exp(-r*t)*(nor_d2_l[0]-nor_d2_l[1])
value_2 = np.exp(-r*t)*(k2-k1)*(nor_d2_l[1]-nor_d2_l[2])
value_3 = (k2-k1)/(k4-k3)*(k4*np.exp(-r*t)*(nor_d2_l[2]-nor_d2_l[3])-p*np.exp(-q*t)*(nor_d1_l[2]-nor_d1_l[3]))
ans = value_1 + value_2 +value_3
print("option value : ",ans)

Enter current price of underlying asset : 15
Enter interest rate : 0.1
Enter q : 0
Enter standard deviation : 0.1
maturity from now (year) : 1
strike price 1 : 20
strike price 2 : 25
strike price 3 : 30
strike price 4 : 35
option value :  0.019312289321292146


In [33]:
# 以蒙地卡羅計算四個執行價選擇權的價格 - 直接以 ND(2.6830,0.25)random期末價格
from scipy.stats import norm
import math
import numpy as np
"""
p1 = float(input("Enter current price of underlying asset : "))
r1 = float(input("Enter interest rate : "))
q1 = float(input("Enter q : "))
sd1 = float(input("Enter standard deviation : "))
t1 = float(input("maturity from now (year) : "))
k11 = float(input("strike price 1 : "))
k21 = float(input("strike price 2 : "))
k31 = float(input("strike price 3 : "))
k41 = float(input("strike price 4 : "))
"""
p1 = 15
r1 = 0.1
q1 = 0
sd1 = 0.5
t1 = 1
k11 =10
k21 = 12
k31 =15
k41 =18

return_l = []
ans_l = []
loc = math.log(p1)+(r1-q1-((sd1**2)/2))*t1
scale = (sd1**2)*t1
print("loc",loc)
print("scale",scale)
size = 1000
for i in range(20):
    rd_sample_l = []
    return_l = []
    for count_t in range(10):
        r = norm.rvs(loc= loc, scale= scale**(1/2),size=1000) #scale是標準差
        for j in range(size):
            rd_sample_l.append(np.exp(float(r[j])))
        for h in range(size):
            if rd_sample_l[h] >= k11 and k21 > rd_sample_l[h]:
                return_l.append(rd_sample_l[h] - k11)
            elif rd_sample_l[h] >= k21 and k31 >= rd_sample_l[h]:
                return_l.append(k21 - k11)
            elif rd_sample_l[h] >= k31 and k41 > rd_sample_l[h]:
                return_l.append(k41 - rd_sample_l[h])
            else:
                return_l.append(0)
    #print(len(return_l))
    ans_l.append(np.mean(return_l)*np.exp(-r1*t1))
ans_mean = np.mean(ans_l)
ans_std = np.std(ans_l)
print("price : ",ans_mean)
print("95% interval : ",ans_mean- 2*ans_std,ans_mean+2*ans_std)

loc 2.68305020110221
scale 0.25
price :  0.6186446519680092
95% interval :  0.5697101266691468 0.6675791772668717


In [32]:
# 以蒙地卡羅計算四個執行價選擇權的價格 2 - EXP(LN(S)+(interest_rate-0.5*σ*σ)*T+σ*SQRT(T)*NORMSINV(RAND())) 老師作法
from scipy.stats import norm
import math
import numpy as np
"""
p1 = float(input("Enter current price of underlying asset : "))
r1 = float(input("Enter interest rate : "))
q1 = float(input("Enter q : "))
sd1 = float(input("Enter standard deviation : "))
t1 = float(input("maturity from now (year) : "))
k11 = float(input("strike price 1 : "))
k21 = float(input("strike price 2 : "))
k31 = float(input("strike price 3 : "))
k41 = float(input("strike price 4 : "))
"""
p1 = 15
r1 = 0.1
q1 = 0
sd1 = 0.5
t1 = 1
k11 =10
k21 = 12
k31 =15
k41 =18

return_l = []
ans_l = []
mean = math.log(p1)+(r1-q1-((sd1**2)/2))*t1
var = (sd1**2)*t1
print("mean",mean)
print("var",var)
size = 10000
for i in range(20):
    rd_sample_l = []
    return_l = []
    for count_t in range(1):
        r = np.random.standard_normal(10000) #scale是標準差
        for j in range(size):
            rd_sample_l.append(np.exp(mean+float(r[j])*(var**0.5)))
        for h in range(size):
            if rd_sample_l[h] >= k11 and k21 > rd_sample_l[h]:
                return_l.append(rd_sample_l[h] - k11)
            elif rd_sample_l[h] >= k21 and k31 >= rd_sample_l[h]:
                return_l.append(k21 - k11)
            elif rd_sample_l[h] >= k31 and k41 > rd_sample_l[h]:
                return_l.append(k41 - rd_sample_l[h])
            else:
                return_l.append(0)
    #print(len(return_l))
    ans_l.append(np.mean(return_l)*np.exp(-r1*t1))
ans_mean = np.mean(ans_l)
ans_std = np.std(ans_l)
print("price : ",ans_mean)
print("95% interval : ",ans_mean- 2*ans_std,ans_mean+2*ans_std)

mean 2.68305020110221
var 0.25
price :  0.6280024854683551
95% interval :  0.6129538981043099 0.6430510728324004
